In [2]:
import numpy as np
import pandas as pd
import nltk
import os
import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import re

from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity

from gensim import corpora, models
from nltk.corpus import stopwords
np.random.seed(42)
from sklearn.metrics import silhouette_score

from transformers import pipeline


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kshamapatel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kshamapatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kshamapatel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import torch

ModuleNotFoundError: No module named 'torch'

In [4]:
!pip install torch --upgrade

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/96/4e/970cd3e13ad95aed81102272f0678d8cc48101880b8be5bae8aad22e7f3b/torch-2.2.0-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 6.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


In [3]:
print(torch.__version__)

1.13.1+cu117


In [4]:
df = pd.read_excel('Winter 2024 Scotia DSD Data Set.xlsx')

In [5]:
df['processed_review'] = df['Review'].apply(lambda x: re.sub('[^a-zA-Z0-9 ]', '', str(x)).lower())


In [6]:
df['word_count'] = df['processed_review'].str.split().apply(len)
short_reviews = df[df['word_count'] <= 3]
df = df[df['word_count'] > 3]
short_reviews

,Review_ID,Date,Rating,Review_Language,Version,Review_Likes,Review,processed_review,word_count
1,1,2023-03-25 19:10:42,5,en,20.47.0,0,App is great.,app is great,3
4,4,2023-11-29 13:54:17,1,en,2310.0.1,0,Sucks,sucks,1
7,7,2021-01-29 20:58:49,5,en,20.21.1,0,Yup,yup,1
10,10,2021-01-04 05:21:00,1,en,20.21.1,0,THIS IS TERABLE,this is terable,3
15,15,2023-08-22 03:00:41,1,en,NaN,0,Who cares???,who cares,2
...,...,...,...,...,...,...,...,...,...
9155,9155,2022-05-11 17:33:43,4,en,20.37.2,0,Not too bad,not too bad,3
9159,9159,2021-01-24 18:52:32,5,en,20.21.1,0,Easy to use,easy to use,3
9163,9163,2023-06-09 22:45:29,4,en,2305.1.0,0,Good service,good service,2
9166,9166,2021-12-07 16:43:51,5,en,20.27.0,0,fantastic,fantastic,1


In [7]:
few_shot_examples = [


    {"sequence": "Worst bank on the planet. Liars, cheats, and theifs, they sell your info to scammers, they get frauded more then any other bank, do your research.. they told me my account was closed and i found out a week later was still active.. they refuse to close it.. they shouldn't be allowed to have a business at all..", "label": "relevant"},
    {"sequence": "Barely works. Barely. Stopped giving out notifications after most recent update and now 2SV doesn't work 👍", "label": "relevant"},
    {"sequence": "The cheque deposit is literally the worst. It used to work no problem and now it never works. Please fix this.", "label": "relevant"},
    {"sequence": "Can't even log on to the app now. Input my password info and it just loads for minutes on end, never getting to the 2SV step. Great work Scotia.", "label": "relevant"},
    {"sequence": "I have been using this app for almost a year now. So far never faced a prob, though it goes out of service multiple times in a week.", "label": "relevant"},
    {"sequence": "Still broken. Can't make payments. Says first time making online payments? We can help with that and then gets an error and says to call them. I called and they couldn't help. It's not my first time making payments, it worked fine until about a year ago. And this garbage app still isn't fixed.", "label": "relevant"},
    {"sequence": "Like many others, I can't get the cheque deposit picture feature to work. Even more frustrating, I can't get the auto capture function to turn off.", "label": "relevant"},
    {"sequence": "Amazingly easy. I love the prompts while taking the photos and the auto shoot feature. Well done!", "label": "relevant"},
    {"sequence": "I have had it with this horrible, untrustworthy, unreliable bank. I will be closing my account asap.", "label": "relevant"},
    {"sequence": "No Longer let's me sign in. Scotiabank can't even figure it out. It's an app issue.", "label": "relevant"},
    {"sequence": "Cheque deposit works now. The recent problem is in regard to the minimum amount required in the momentum account. Bill payments often come out near midnight and sometimes that brings the account below the minimum. Then you get charged the account fee because you're below the minimum. The amount in the account should be calculated at a certain time (end of business day perhaps) rather than at midnight on the calendar day", "label": "relevant"},
    {"sequence": "I can't transfer money between my accounts on either my mobile app or when access through web browser on my mobile. Also, when I sign in to my Scotia app I cant log on until I sign on to the two steps password.", "label": "relevant"},
    {"sequence": "Highly unstable and poorly designed app. With tons of bugs and instability.", "label": "relevant"},
    {"sequence": "The app won't open on my Samsung galaxy s20 Fe", "label": "relevant"},
    {"sequence": "Has alot of glitches. I'd transfer money into another account and my total would show double the money. Also the nightly maintenance is hella annoying. I can't access certain accounts or do any transfers while its going on.", "label": "relevant"},
    {"sequence": "App is trash, I always spend 20 mins plus trying to deposit a cheque by taking a photo. Always on a dark surface, well lit, all corners in frame, everything it states to do but yet it can never deposit. I always end up having to go to an atm or in branch, what's the point of the feature. Super frustrating...", "label": "relevant"},
    {"sequence": "App kept crashing when I tried e-transfer. Uninstalled and reinstalled. Put in proper credentials. Says sent 2 step verification to phone but never got a notification. Checked phone ALL notifications allowed. Still no 2 step. App is garbage waste of time!! It's now a month or so later still same garbage force us to use 2SV and an app that doesn't even work properly. Thanks a lot Scotiabank after 35+ years of a great service, you had to turn it into garbage!!!", "label": "relevant"},
    {"sequence": "I find it absolutely terrible. I ha e other bank accounts no problems. SB never seems to want to see the chq through my phone. Never had problems until the last fee months. Can take me 30 mins to keep trying to deposit a chq. I HATE It.", "label": "relevant"},
    {"sequence": "Annoying frequent and required password resets.", "label": "relevant"},
    {"sequence": "Doesnt work as well as other banks and always has errors or check later or not available now issues....", "label": "relevant"},
    {"sequence": "Takes forever to log on ever since they change to use fingerprints. Slow and unusable.", "label": "relevant"},
    {"sequence": "Very homie ! I stay every month and don't plan on ever changing. Plenty of food options walking distance.", "label": "irrelevant"},
    {"sequence": "Good value for the price thought that the breakfast could be a little more plentiful compared to other hotel breakfasts", "label": "irrelevant"},
    {"sequence": "Older hotel, down town location which was inconvenient from the highway. Rooms were really clean, comfy beds but breakfast was limited for a hot breakfast.", "label": "irrelevant"},
    {"sequence": "when returning to the palm beach, dubai would not consider any other accomodation and willmseek other Holiday Innth,s when travelling", "label": "irrelevant"},
    {"sequence": "Very good facility.  Pleased with everything. Breakfast was really nice.  Location was very good near restaurants in the area.      ", "label": "irrelevant"},
    {"sequence": "Various members of the staff went out of there way to make sure our request  during our four day stay.", "label": "irrelevant"},
    {"sequence": "wonderful view of the Locks and perfect location to be able to walk to everything downtown. loved the price!", "label": "irrelevant"},
    {"sequence": "Very close to city hospital for out of towners.Clean and close to shopping and the airport.Have been back twice.", "label": "irrelevant"},
    {"sequence": "This was our third time staying. Always clean and quite. Will stay again and would recommend this hotel.", "label": "irrelevant"},
    {"sequence": "Very nice hotel, easy to find, nice breakfast and clean room.  Would be nice to have another chair in the room. ", "label": "irrelevant"},
    {"sequence": "The rooms smelled fine, the bathrooms were sufficient and breakfast was better than some other 'pricy' motels. I will stay there again. Oh.....I slept great!", "label": "irrelevant"},
    {"sequence": "The location and the historical feel of the Ojibway are great, but the hotel is in dire need of a facelift.", "label": "irrelevant"},
    {"sequence": "An excellent, most comfortable pillows Ever, courteous place to stay! Feels like being home yet waited on!!!!! Amazing", "label": "irrelevant"},
    {"sequence": "Great service, comfy beds, clean contemporary rooms. Exactly what I needed for a quick stop through city Ste . Thank you", "label": "irrelevant"},
    {"sequence": "We were just passing through SSM and this hotel was exactly what we needed. Clean and comfortable rooms. Service was exemplary!", "label": "irrelevant"},
    {"sequence": "Koodos to this establishment. The staff go over and beyond to make one,s stay very comfortable as well pleasant. Would go back to this place, without any hesitation.", "label": "irrelevant"},
    {"sequence": "The lady had the counter was friendly and recommended a near dog park for my dog.The room was clean and I had no problem.", "label": "irrelevant"},
    {"sequence": "Stopover on a road trip. This is a very clean and modern hotel. It is well located. The staff were friendly  and helpful.", "label": "irrelevant"},
    {"sequence": "It was a nice convient place to stay with the trip we were travelling in mind.  Good quality to price ratio.", "label": "irrelevant"},
    {"sequence": "It don't work half the time and limited options with a mess of a menu.", "label": "irrelevant"},
    {"sequence": "wonderful service good bonuses nice", "label": "other"},
    {"sequence": "a breeze to use", "label": "other"},
    {"sequence": "gets the job done i guess", "label": "other"},
    {"sequence": "great convienent services with online banking", "label": "other"},
    {"sequence": "muy clara y buensima", "label": "other"},
    {"sequence": "its under construction alot", "label": "other"},
    {"sequence": "your services are amazing to humanity", "label": "other"},
    {"sequence": "it was great of mr jagrajkang really appreciate his work", "label": "other"},
    {"sequence": "great love that girl from the dominican republic", "label": "other"},
    {"sequence": "always in good hands here", "label": "other"},
    {"sequence": "Excellent access, easy to use and Claire.", "label": "other"},
    {"sequence": "Poor management poor service. Need an improvement", "label": "other"},
    {"sequence": "Very simple to navigate.", "label": "other"},
    {"sequence": "very easy and you can do it without living your home.", "label": "other"},
    {"sequence": "Sleek, easy to use design.", "label": "other"},
    {"sequence": "Solid and stable. Zero complaints.", "label": "other"},
    {"sequence": "Very easy to navigate", "label": "other"},
    {"sequence": "Pretty close to the computer experience", "label": "other"},
    {"sequence": "Poorly designed for seniors citizens.", "label": "other"},
    {"sequence": "love my local branch, great people and always helpful", "label": "other"}
]


In [8]:
df_few_shot_examples = pd.DataFrame(few_shot_examples)

In [9]:
df_few_shot_examples

,sequence,label
0,"Worst bank on the planet. Liars, cheats, and t...",relevant
1,Barely works. Barely. Stopped giving out notif...,relevant
2,The cheque deposit is literally the worst. It ...,relevant
3,Can't even log on to the app now. Input my pas...,relevant
4,I have been using this app for almost a year n...,relevant
5,Still broken. Can't make payments. Says first ...,relevant
6,"Like many others, I can't get the cheque depos...",relevant
7,Amazingly easy. I love the prompts while takin...,relevant
8,"I have had it with this horrible, untrustworth...",relevant
9,No Longer let's me sign in. Scotiabank can't e...,relevant


In [10]:
# !pip install torch --upgrade

In [11]:
# ! pip install torch==2.1.0

In [12]:
!pip install git+https://github.com/pmbaumgartner/setfit -q


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# import torch

In [13]:
from setfit import SetFitClassifier

In [14]:
clf = SetFitClassifier("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [24]:
# clf = SetFitClassifier("stsb-mpnet-base-v2")

In [15]:
review = df_few_shot_examples['sequence'].to_list()
label = df_few_shot_examples['label'].to_list()

In [17]:
# !pip install torch

In [16]:
clf.fit(review,label)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

In [17]:
df['predicted'] = clf.predict(df['Review'].to_list())

In [23]:
filtered_reviews1 = df[df['predicted'] == 'other']
filtered_reviews2 = df[df['predicted'] == 'irrelevant']
filtered_reviews3 = df[df['predicted'] == 'relevant']

In [36]:
r3 = filtered_reviews2[~filtered_reviews2['processed_review'].str.contains('app|bank|banking|scotiabank|2sv|scotia|account|Appointment|2', case=False)]

In [48]:
r3.to_csv('/content/EASTER_EGG.csv', index=False)

In [37]:
r3

,Review_ID,Date,Rating,Review_Language,Version,Review_Likes,Review,processed_review,word_count,predicted,GMM_Pred
20,20,2021-03-01 10:56:52,5,en,20.24.0,0,Very homie ! I stay every month and don't plan...,very homie i stay every month and dont plan o...,18,irrelevant,0
34,34,2021-11-15 10:49:54,4,en,20.32.0,0,Good value for the price thought that the brea...,good value for the price thought that the brea...,20,irrelevant,0
35,35,2022-06-27 21:08:31,5,en,20.39.1,0,"Sleek, easy to use design.",sleek easy to use design,5,irrelevant,0
41,41,2023-03-23 12:53:22,4,en,20.47.0,0,"Older hotel, down town location which was inco...",older hotel down town location which was incon...,25,irrelevant,0
68,68,2021-08-18 13:14:40,5,en,20.29.0,0,"when returning to the palm beach, dubai would ...",when returning to the palm beach dubai would n...,20,irrelevant,0
...,...,...,...,...,...,...,...,...,...,...,...
9031,9031,2023-03-22 13:35:10,1,en,20.47.0,0,The bathrooms looked clean but the building wa...,the bathrooms looked clean but the building wa...,27,irrelevant,0
9068,9068,2021-07-01 11:29:50,5,en,20.28.0,0,Very clean comfortable motel and well situated...,very clean comfortable motel and well situated...,21,irrelevant,0
9078,9078,2021-04-15 13:12:52,4,en,20.25.1,0,"i had to stay in the palm beach, dubai for the...",i had to stay in the palm beach dubai for the ...,25,irrelevant,0
9137,9137,2023-04-23 13:57:22,2,en,20.48.0,0,"Really old. Wi-Fi not good, my son has to go u...",really old wifi not good my son has to go up t...,27,irrelevant,0


In [26]:
filtered_reviews3

,Review_ID,Date,Rating,Review_Language,Version,Review_Likes,Review,processed_review,word_count,predicted
0,0,2022-04-20 11:38:29,1,en,20.37.2,0,"Worst bank on the planet. Liars, cheats, and t...",worst bank on the planet liars cheats and thei...,58,relevant
2,2,2022-05-31 00:54:40,1,en,20.38.1,0,Barely works. Barely. Stopped giving out notif...,barely works barely stopped giving out notific...,16,relevant
3,3,2021-06-18 13:16:44,5,en,20.27.0,0,Really easy for a technophobe,really easy for a technophobe,5,relevant
5,5,2021-09-19 12:32:37,5,en,20.30.1,0,"Now I can use my fingerprints to log on, just ...",now i can use my fingerprints to log on just m...,15,relevant
6,6,2022-04-09 11:49:39,1,en,20.37.1,0,The cheque deposit is literally the worst. It ...,the cheque deposit is literally the worst it u...,21,relevant
...,...,...,...,...,...,...,...,...,...,...
9170,9170,2023-07-23 11:57:08,2,en,2306.0.2,0,services are often unavailable.,services are often unavailable,4,relevant
9171,9171,2021-04-20 10:16:28,5,en,20.25.1,0,Great app. Easy to use!,great app easy to use,5,relevant
9172,9172,2023-03-05 10:27:12,5,en,20.47.0,0,Great App Top Notch Top Notch!,great app top notch top notch,6,relevant
9173,9173,2023-03-14 15:28:08,5,en,20.47.0,0,It works like this to should,it works like this to should,6,relevant


In [27]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [28]:
# filtered_reviews = filtered_reviews[['processed_review']]

In [53]:
np.random.seed(42)
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X = vectorizer.fit_transform(filtered_reviews3['processed_review']).toarray()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


pca = PCA(n_components=2,svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

# Gaussian Mixture
gmm = GaussianMixture(n_components=20)
gmm_preds = gmm.fit_predict(X_pca)

filtered_reviews3['GMM_Pred'] = gmm_preds

<ipython-input-53-26abc7896c2a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews3['GMM_Pred'] = gmm_preds


In [54]:
cluster_counts = filtered_reviews3['GMM_Pred'].value_counts()

print("Cluster Counts:")
print(cluster_counts)

Cluster Counts:
6     694
0     552
18    522
10    429
12    382
16    352
1     349
11    340
8     313
7     313
14    246
3     218
4     173
5     159
19    149
17    141
15    136
13     86
2      74
9      56
Name: GMM_Pred, dtype: int64


In [55]:
r4 = filtered_reviews3[(filtered_reviews3['GMM_Pred'] == 9)]
r4

,Review_ID,Date,Rating,Review_Language,Version,Review_Likes,Review,processed_review,word_count,predicted,GMM_Pred
24,24,2022-01-07 09:25:22,3,en,20.34.3,1,"Like many others, I can't get the cheque depos...",like many others i cant get the cheque deposit...,26,relevant,9
157,157,2022-11-23 14:21:40,2,en,20.44.2,3,Depositing a cheque is impossible. I have a Sa...,depositing a cheque is impossible i have a sam...,93,relevant,9
431,431,2023-06-11 22:24:05,1,en,2305.1.0,31,Edit* App is still absolutely TERRIBLE for dep...,edit app is still absolutely terrible for depo...,82,relevant,9
453,453,2022-01-06 21:19:56,1,en,20.34.3,3,Mobile deposit took 6 attempts to complete. Ke...,mobile deposit took 6 attempts to complete kep...,36,relevant,9
477,477,2022-01-09 02:59:48,2,en,20.34.3,0,"The ""Deposit a cheque"" camera feature is compl...",the deposit a cheque camera feature is complet...,30,relevant,9
656,656,2023-03-09 10:57:33,1,en,20.47.0,0,Worst update yet... Today alone I have attempt...,worst update yet today alone i have attempted ...,66,relevant,9
1054,1054,2022-09-14 10:33:36,1,en,20.42.1,12,"Months after the update, problems with deposit...",months after the update problems with depositi...,83,relevant,9
1064,1064,2022-01-14 10:19:13,1,en,20.34.4,1,Despite saying the cheque capture was going to...,despite saying the cheque capture was going to...,32,relevant,9
1081,1081,2021-12-23 15:57:21,1,en,20.34.3,24,Deposit a cheque feature use to be fantastic t...,deposit a cheque feature use to be fantastic t...,75,relevant,9
1193,1193,2022-01-31 13:05:40,1,en,20.35.0,1,I can not get the cheque to deposit no matter ...,i can not get the cheque to deposit no matter ...,34,relevant,9


In [56]:
topics_importance =[
    {"sequence": "The 2SV feature adds an extra layer of security, which is great. However, sometimes the code takes too long to arrive, causing delays in accessing my account.","label": "2SV"},
    {"sequence": "I appreciate the security that the two-step verification provides, but I wish there was an option to remember trusted devices so I don't have to do it every time.","label": "2SV"},
    {"sequence": "Two-Step Verification is a must for online banking, and this app does it well. The only issue is the app sometimes crashes after entering the code.","label": "2SV"},
    {"sequence":  "The numerical code for 2SV sometimes fails to send, which is frustrating. When it works, it's seamless and quick, but reliability needs improvement.","label": "2SV"},
    {"sequence": "Great security feature with the 2SV, but it would be better if the app allowed the use of third-party authenticators as an option.","label": "2SV"},
    {"sequence": "I find the 2SV process a bit tedious, especially when I'm in a hurry. Maybe add a feature to expedite the process for frequent logins from the same device?","label": "2SV"},
    {"sequence": "Love the extra security with the two-step verification. However, the app doesn't always send the code on the first try, which can be annoying.","label": "2SV"},
    {"sequence":  "Two-Step Verification works, but the interface could be more user-friendly. Instructions on setting it up were not very clear for less tech-savvy users.","label": "2SV"},
    {"sequence": "The app's 2SV is reliable, but I've experienced a few instances where the code was delayed by several minutes, making it a bit inconvenient.","label": "2SV"},
    {"sequence": "Overall, the two-step verification is a solid security feature. However, the app could use some optimization to make the process smoother and faster.","label": "2SV"}
]

In [57]:
sample_reviews_01 = [
    "I've been using this banking app for a few months. While it's generally reliable, I've noticed some lag when switching between accounts. It's not a deal-breaker, but it can be frustrating at times, especially during busy hours.",
    "The app's performance is quite good on my new phone. Transactions are smooth and the app opens quickly. However, on my older tablet, it's a different story – it's pretty slow and sometimes crashes.",
    "I'm impressed with the speed of this app. Transfers and payments are completed almost instantly. However, I've experienced a few crashes after the last update, which is concerning.",
    "Overall, the app works well, but it tends to slow down when I'm checking my transaction history. It's not a major issue, but it can be annoying when I'm in a hurry.",
    "The app's interface is user-friendly, but the performance is hit or miss. Sometimes it's fast, other times it takes ages to load my account details. Consistency would be appreciated.",
    "This banking app has been a lifesaver for quick transactions. However, I've noticed it can be sluggish when the network is weak, even when other apps work fine under the same conditions.",
    "Performance-wise, this app is a mixed bag. On good days, it's a breeze to use, but there are days when it just refuses to cooperate, taking forever to load or update my balance.",
    "I like how the app is designed, but it freezes occasionally, especially when I try to access my loan details. It needs some optimization to make it more reliable.",
    "The recent update improved the app significantly. It's faster and more responsive now, but there's still a slight delay when I switch between different sections of the app.",
    "Decent app for everyday banking, but I've faced issues with the fingerprint login. It often takes multiple attempts to recognize my fingerprint, which can be quite time-consuming."
]
# 2SV
sample_reviews_00 = [
    "The 2SV feature adds an extra layer of security, which is great. However, sometimes the code takes too long to arrive, causing delays in accessing my account.",
    "I appreciate the security that the two-step verification provides, but I wish there was an option to remember trusted devices so I don't have to do it every time.",
    "Two-Step Verification is a must for online banking, and this app does it well. The only issue is the app sometimes crashes after entering the code.",
    "The numerical code for 2SV sometimes fails to send, which is frustrating. When it works, it's seamless and quick, but reliability needs improvement.",
    "Great security feature with the 2SV, but it would be better if the app allowed the use of third-party authenticators as an option.",
    "I find the 2SV process a bit tedious, especially when I'm in a hurry. Maybe add a feature to expedite the process for frequent logins from the same device?",
    "Love the extra security with the two-step verification. However, the app doesn't always send the code on the first try, which can be annoying.",
    "Two-Step Verification works, but the interface could be more user-friendly. Instructions on setting it up were not very clear for less tech-savvy users.",
    "The app's 2SV is reliable, but I've experienced a few instances where the code was delayed by several minutes, making it a bit inconvenient.",
    "Overall, the two-step verification is a solid security feature. However, the app could use some optimization to make the process smoother and faster."
]

# Application Performance
sample_reviews_01 = [
    "I've been using this banking app for a few months. While it's generally reliable, I've noticed some lag when switching between accounts. It's not a deal-breaker, but it can be frustrating at times, especially during busy hours.",
    "The app's performance is quite good on my new phone. Transactions are smooth and the app opens quickly. However, on my older tablet, it's a different story – it's pretty slow and sometimes crashes.",
    "I'm impressed with the speed of this app. Transfers and payments are completed almost instantly. However, I've experienced a few crashes after the last update, which is concerning.",
    "Overall, the app works well, but it tends to slow down when I'm checking my transaction history. It's not a major issue, but it can be annoying when I'm in a hurry.",
    "The app's interface is user-friendly, but the performance is hit or miss. Sometimes it's fast, other times it takes ages to load my account details. Consistency would be appreciated.",
    "This banking app has been a lifesaver for quick transactions. However, I've noticed it can be sluggish when the network is weak, even when other apps work fine under the same conditions.",
    "Performance-wise, this app is a mixed bag. On good days, it's a breeze to use, but there are days when it just refuses to cooperate, taking forever to load or update my balance.",
    "I like how the app is designed, but it freezes occasionally, especially when I try to access my loan details. It needs some optimization to make it more reliable.",
    "The recent update improved the app significantly. It's faster and more responsive now, but there's still a slight delay when I switch between different sections of the app.",
    "Decent app for everyday banking, but I've faced issues with the fingerprint login. It often takes multiple attempts to recognize my fingerprint, which can be quite time-consuming."
]


# Accessibility
sample_reviews_02 = [
    "I appreciate the voice command feature for transactions, making it easier for those with visual impairments like myself. However, the app often misunderstands my commands, leading to frustrating experiences.",
    "As someone with limited mobility, I find the large buttons helpful. But the app lacks clear instructions for first-time users, which can be confusing.",
    "The app's text-to-speech is a great help for visually impaired users. But it's not available for all sections of the app, which is a bit disappointing.",
    "As a hearing-impaired user, I rely on visual cues. This app has clear and readable alerts, but the color contrast is too low, making it hard to read under certain lighting conditions.",
    "I have cognitive disabilities and find the simple layout of this app quite user-friendly. But the lack of a consistent design across different pages sometimes makes it hard to navigate.",
    "The recent update improved the screen reader compatibility, which is great for blind users like myself. However, the app crashes occasionally, which can be quite inconvenient.",
    "I use a wheelchair and have limited hand dexterity. The app's gesture controls are a great idea, but they require too much precision, which can be challenging for me.",
    "The app's font size can be adjusted, which is helpful for my poor vision. But the font style itself is not very readable for me, especially in longer texts.",
    "As someone with a cognitive impairment, I find the step-by-step instructions for transactions very helpful. However, the app sometimes runs slow, which can be confusing if you're waiting for a page to load.",
    "The haptic feedback feature is useful for those who can't rely on visual or audio cues. But it's not consistent across the app, which can lead to missed notifications."
]


# Appointment Booking
sample_reviews_03 = [
    "I tried booking an appointment with an advisor using the app, but it kept crashing. Managed to book after a few tries, though.",
    "The appointment booking feature is a great addition, but it needs more time slots. It's hard to find a convenient time.",
    "Booking an appointment was straightforward, but the confirmation email arrived late, which was confusing.",
    "I appreciate the convenience of booking appointments through the app, but the interface could be more user-friendly.",
    "Successfully booked an appointment, but the advisor was not well-prepared for my questions. The app should ensure advisors review our profiles first.",
    "The appointment booking process is easy, but I wish there were options for virtual meetings as well.",
    "I love the ease of booking appointments, but the app doesn't always save my preferred branch location, which is annoying.",
    "The app makes booking appointments with banking advisors hassle-free, but it sometimes shows no available slots for weeks.",
    "Great feature for booking appointments, but the lack of immediate confirmation makes me unsure if it went through.",
    "Booking an appointment is easy, but I had to reschedule, and the app didn't update my appointment time promptly."
]


# Biometric Login
sample_reviews_04 = [
    "The biometric login is super convenient. It's quick and recognizes my fingerprint almost instantly every time. A real time-saver!",
    "I've had mixed experiences with the biometric login. It works smoothly most of the time, but occasionally it doesn't recognize my fingerprint, which can be frustrating.",
    "Biometric login is a game changer for me. I no longer have to remember complex passwords, and it feels much more secure.",
    "The app's biometric login feature is good but not perfect. Sometimes it's slow to respond, and I have to attempt it a couple of times before it works.",
    "I'm impressed with the biometric login on this app. It's fast and efficient, making my banking experience much smoother.",
    "Not a fan of the biometric login. It often fails to recognize my fingerprint, and I end up using the password method anyway.",
    "The fingerprint login is a convenient feature, but I've noticed it drains my phone's battery faster. It's a trade-off between convenience and battery life.",
    "Biometric login on this app is hit or miss for me. When it works, it's fantastic, but too often, it doesn't, which is annoying.",
    "Love the fingerprint login feature! It's quick and feels more secure than typing a password. Great addition to the app.",
    "The app's biometric login feature needs improvement. It's not as responsive as I'd like, and sometimes it takes multiple attempts to log in."
]


# Budgeting
sample_reviews_05 = [
    "The budgeting tool is quite helpful, but it can be glitchy at times. The spending graphs are a nice touch, though they don't always update in real-time.",
    "I love the budgeting feature on this app! It's been a game-changer in tracking my expenses. However, I wish the categories were more customizable.",
    "Overall, the budgeting feature is decent, but it lacks flexibility. For instance, I can't create my own categories for expenses, which is frustrating.",
    "The app's budgeting function is user-friendly, but the accuracy of the spending tracker needs improvement. Sometimes, my transactions are misclassified.",
    "I'm impressed with the detailed spending graphs in the budgeting section. They really help me see where my money goes, though the app can be slow to load them.",
    "The budgeting feature is okay, but it's very basic. I would appreciate more advanced options, like setting different budgets for each category.",
    "Great concept for budgeting, but the execution is lacking. The app often crashes when I try to access the budgeting tool, which is annoying.",
    "I like how the budgeting feature helps me track my income and expenses, but it's not very intuitive. New users might find it hard to navigate.",
    "The budgeting tool could be more accurate. It sometimes double-counts expenses, which throws off my entire budget. Needs some fixes.",
    "Budgeting with this app is simple and straightforward, but it could use more in-depth features, like forecasting future expenses based on past trends."
]


# Chat
sample_reviews_06 = [
    "I recently used the chat feature in the app to resolve an issue with my account. The chatbot was initially helpful, but it couldn't understand my specific problem. Eventually, I was transferred to a live agent who was very knowledgeable and solved my issue quickly.",
    "I'm impressed with the chat option in the banking app. It's convenient to have a quick chat with a live person rather than waiting on hold on the phone. The responses are prompt and the staff is always friendly.",
    "The chatbot is a nice touch, but it needs improvement. It often misinterprets what I'm saying, leading to frustrating interactions. However, once I get through to a human, the service is excellent.",
    "I love the chat feature! It's so much easier to just send a message in the app rather than calling and waiting. The chatbot is surprisingly good at answering basic questions, which saves a lot of time.",
    "The chat option is a hit and miss. Sometimes I get my queries resolved quickly, other times I end up going in circles with the chatbot. There should be an easier way to reach a human representative.",
    "Not a fan of the chat feature. The chatbot is almost useless for anything beyond basic questions, and it takes forever to get connected to a human. It's faster to just call customer service.",
    "The chat function in the app is a game changer. I had an issue with a transaction and the chatbot quickly directed me to the right department. The live agent was very helpful and resolved my issue in no time.",
    "The chat feature is okay, but it could be better. The chatbot is not very intuitive and often redirects me to FAQs instead of providing real answers. But once I reach a live agent, they are usually helpful.",
    "I had a great experience using the chat feature. The chatbot quickly understood my query and provided a clear answer. When I needed more detailed help, switching to a live agent was seamless.",
    "The chatbot is a good idea, but it's not there yet. It struggles with complex queries and often transfers me to a live agent, which takes additional time. The live agents are great, but I wish the bot was more capable."
]


# Cheque Deposit
sample_reviews_07 = [
    "I've been using the cheque deposit feature for a few months now. It's convenient, but sometimes the app has trouble reading the cheque details, and I have to enter them manually.",
    "The cheque deposit function is a lifesaver! No more trips to the bank. However, it does take a couple of days for the funds to show up in my account, which can be a bit frustrating.",
    "I appreciate the cheque deposit feature in this app, but it's not very intuitive for first-time users. I had to look up a tutorial to understand how to properly align the cheque for a picture.",
    "The app's cheque deposit is generally good, but it once failed to process a cheque correctly and I had to visit the branch. It's mostly reliable, but there's room for improvement.",
    "Love the convenience of depositing cheques through my phone! The only downside is the app's camera function sometimes doesn’t capture a clear image, and I have to retake the photo several times.",
    "Cheque deposit feature works well most of the time, but I've had issues with larger cheques not being accepted through the app, which can be a bit of a hassle.",
    "This feature has made my life so much easier, no more waiting in lines! However, the app could do better with providing clearer instructions on how to use this feature effectively.",
    "I was excited about the cheque deposit feature, but it’s quite slow and sometimes the app crashes while processing the cheque, which is quite annoying.",
    "The cheque deposit is a decent feature, but the daily deposit limit is too low for my needs. It would be great if the bank could consider increasing the limit.",
    "Overall, the cheque deposit feature is a great idea, but the app often takes a long time to verify the cheque, which can be inconvenient when I'm in a hurry."
]


# Credit Score
sample_reviews_08 = [
    "I've been using this app for a few months, and I'm pretty happy with the credit score feature. It updates regularly, but sometimes the score seems a bit off compared to other sources.",
    "The credit score feature on this app is a game changer for me. It's easy to understand and helps me track my credit health. However, the app occasionally crashes when I try to access this feature.",
    "Honestly, I'm a bit disappointed with the credit score feature. It's not as detailed as I expected, and updates are slower than other apps I've used.",
    "Love the credit score feature! It gives a good breakdown of factors affecting my score. But, the app could use a more user-friendly interface.",
    "The credit score feature is decent but lacks explanations for score changes. Also, the app's notifications are a bit too frequent for my liking.",
    "This app's credit score feature is quite accurate and mirrors what I see on official credit reports. However, the overall app design is a bit outdated.",
    "The credit score feature is okay, but it could be more interactive. I'd like to see more tips on improving my score. Also, the app sometimes takes a while to load.",
    "I'm new to this app, and the credit score feature seems useful. But, it's a bit overwhelming for beginners, and customer support isn't very helpful.",
    "The credit score feature is a nice addition, but it doesn't always reflect recent payments or credit changes promptly. Needs some improvement.",
    "Good credit score feature, but it's pretty basic. I wish the app provided more in-depth analysis and suggestions for credit improvement."
]


# Email Money Transfer
sample_reviews_09 = [
    "The Email Money Transfer feature in the Scotiabank app is pretty convenient, but sometimes it's a bit slow. Overall, it's a useful tool for quick transfers.",
    "I had issues with Email Money Transfer. The process was not as intuitive as I expected, and I had to contact support for help. They were responsive, though.",
    "Great feature for transferring money via email, but the daily limit is a bit restrictive. It's good for small transactions but not for larger ones.",
    "I love the Email Money Transfer on the Scotiabank app! It's so easy to send money to my friends. However, the app occasionally crashes mid-transfer.",
    "Email Money Transfer is a lifesaver for paying rent and splitting bills. But, it would be better if the transaction history was more detailed.",
    "The Email Money Transfer function works well most of the time, but I've experienced delays in fund transfer a few times. Needs some improvement.",
    "I appreciate the convenience of Email Money Transfer, but the security measures can be a bit annoying. I understand the need for security, but it sometimes feels excessive.",
    "Email Money Transfer is handy, but the fees for non-Scotiabank transfers seem a bit high compared to other apps. Good for in-bank transfers, though.",
    "The app's Email Money Transfer feature is generally reliable, but I had an issue where the recipient didn't get the notification email. Customer service sorted it out eventually.",
    "Email Money Transfer is a decent feature, but it's not available for international transfers. Would love to see that added in the future."
]


# Errors
sample_reviews_10 = [
    "I've been using this app for a few months, and while it's mostly good, I've noticed it crashes unexpectedly, especially when I try to view my transaction history.",
    "Decent app, but it has a major flaw: sometimes payments don't go through and it doesn't even notify you. Missed a bill payment because of this glitch.",
    "The user interface is great, but the app often logs me out randomly, which is frustrating when I’m in the middle of a transaction.",
    "Generally a reliable app, but it has trouble syncing with my account details. I often see outdated balance info, which is confusing.",
    "The app is okay, but there's a bug where it doesn't always update recent transactions. Makes it hard to keep track of my spending.",
    "Not impressed with this app. It frequently crashes when I try to deposit checks using the camera feature.",
    "The app's budgeting tool is handy, but there's a glitch that sometimes duplicates entries, throwing off my entire budget.",
    "It's a good banking app, but it has its quirks. For instance, the fingerprint login fails randomly, forcing me to use my password.",
    "The app is mostly functional, but I've experienced a few errors when trying to transfer money to friends. It gets stuck on the confirmation screen.",
    "Overall, the app works, but it’s not without its flaws. The chat support feature often doesn't load, leaving me without help."
]


# Fee
sample_reviews_11 = [
    "The app is quite user-friendly, but I'm disappointed with the unexpected maintenance fees. Wasn't mentioned clearly when I signed up.",
    "Generally a good experience, but the transaction fees for international transfers seem a bit steep compared to other banks.",
    "Love the interface and features, but the overdraft fees are a real downside. It would be great if they were more transparent about these charges.",
    "The app works fine, but the monthly account fee just for basic services is a bit too much for my liking.",
    "Good app, easy to navigate. However, the ATM fees for non-network ATMs are higher than I expected.",
    "I appreciate the app's functionality, but the fees for check deposits are somewhat annoying. It diminishes the overall convenience.",
    "Solid app with great features, but the annual fee for the premium account doesn't seem worth it. Needs more benefits to justify the cost.",
    "The app's design is top-notch, but the hidden fees for certain transactions were a nasty surprise. They need better fee disclosures.",
    "I'm happy with the app's performance, but the fees for wire transfers are more than I'd like to pay. Considering other options.",
    "The mobile deposit feature is a lifesaver, but the fee for immediate fund availability is a bit discouraging."
]


# Info Alerts
sample_reviews_12 = [
    "I love the Info Alerts feature on this app. It keeps me updated about all my account activities. However, sometimes the alerts are delayed, which can be a bit concerning.",
    "The Info Alerts are helpful but a bit too frequent for my liking. I appreciate staying informed, but it would be great to have an option to customize the frequency.",
    "Great concept with the Info Alerts, but they don't always come through. Missed a few important notifications which wasn't ideal.",
    "Info Alerts are a lifesaver! They helped me catch unauthorized activity on my account. But, the app interface could be more user-friendly when viewing these alerts.",
    "The alerts are generally good, but I've noticed they're not very detailed. Would be better if they included more specific information about the transactions.",
    "Really like the Info Alerts feature, it's very reassuring. However, the app sometimes sends duplicate notifications which can be annoying.",
    "The Info Alerts keep me well-informed about my account, but the lack of customization options is a drawback. I'd prefer to choose the types of alerts I receive.",
    "Info Alerts are a hit and miss. When they work, they're great, but I've had a few instances where they failed to notify me about significant transactions.",
    "I appreciate the security that Info Alerts provide, but the app could definitely improve in alerting me faster. There's often a noticeable delay.",
    "The Info Alerts feature is decent, but it would be great if the app allowed for more detailed notification settings. Sometimes it feels a bit too generic."
]


# International Money Movement
sample_reviews_13 = [
    "Had a mixed experience with international transfers. The rates are competitive, but the app crashed twice during the process. Took me a whole day to successfully send money to my family in Brazil.",
    "I'm impressed with the ease of setting up international transfers. However, the transaction fees are a bit high compared to other services. Fast delivery though.",
    "Quite disappointed with the transfer limits for international transactions. I understand security concerns, but these limits are too restrictive for my business needs.",
    "The app's user interface for international money movement is intuitive, but I faced issues with verification. Customer service was helpful, but it still took longer than expected to resolve.",
    "Love the feature that allows me to track my international transfer in real time. But, the exchange rates fluctuate a lot, and I ended up paying more than I initially thought.",
    "Successfully used the app for sending money to my relatives in the Philippines. The process was smooth, but I wish the app had better support for different currencies.",
    "The app is generally reliable for international transfers, but last time I had a transaction delayed for no clear reason. It caused quite a bit of inconvenience.",
    "Excellent for small international transfers. But when I tried to send a larger amount, the app required additional verification which took several days. Not ideal for urgent transactions.",
    "The integration with Western Union is a plus, but I experienced some glitches when trying to link my account. Once set up, the transfers were fairly straightforward.",
    "Decent app for international money movement. The fees are okay, but the customer support needs improvement. Had a hard time getting clear answers to my questions about transfer times."
]


# Investments
sample_reviews_14 = [
    "Just started using the investment feature for GICs. The process is straightforward, but I wish there were more options for shorter terms.",
    "Love the RRSP options available through the app. Really easy to manage my retirement savings. However, the app can be slow to update my account balance.",
    "Great range of investment options, but the fees are a bit high compared to other apps. Good for beginners who need guidance though.",
    "The app makes it easy to diversify my portfolio. However, customer service can be slow to respond to investment-related queries.",
    "I like the educational resources about investments, but the actual execution of trades is a bit clunky and could be smoother.",
    "Decent app for managing investments, but it crashes occasionally. Lost a significant opportunity to adjust my GIC because of this.",
    "Really impressed with the variety of RRSP investment options. But, the app's interface could be more user-friendly.",
    "The investment tracking tools are helpful, but I experienced some glitches when trying to transfer funds between accounts.",
    "Solid app for basic investment management, but it lacks advanced features for more experienced investors.",
    "Good for keeping an eye on my investments, but the notifications about market changes are not as timely as I'd like."
]


# Login and Logout Issues
sample_reviews_15 = [
    "Had trouble logging into my account today. Kept getting an error message saying my password was incorrect, but I'm sure it was right. Had to reset my password twice. Quite frustrating!",
    "The app logs me out randomly, often in the middle of a transaction. It's inconvenient and makes me worry about the security of my transactions.",
    "Login was a breeze for me. The face recognition feature works well. But, logging out seems to take a few extra steps than necessary.",
    "Experienced some issues with the two-factor authentication. The app didn't send the verification code to my phone, so I couldn't log in for a whole day.",
    "I appreciate the security measures for login, but the app tends to forget my device frequently. I have to go through device verification almost every week.",
    "No major issues with login, but the app often crashes when I try to log out, which is quite annoying.",
    "The biometric login is a hit or miss for me. Sometimes it logs me in quickly, other times I have to enter my password manually.",
    "Logging in is generally fine, but the app keeps logging me out if I switch to another app for a minute. It's quite inconvenient to log in repeatedly during a single session.",
    "I've been locked out of my account several times after the app failed to recognize my correct password. Customer service was helpful but it's a recurring issue.",
    "The logout button is not very prominent in the app, which is a minor issue but can be improved for better user experience."
]


# Quick Balance
sample_reviews_16 = [
    "Love the Quick Balance feature! It's so convenient to check my balance on the go. However, it sometimes takes a while to update, which can be frustrating.",
    "Quick Balance is a lifesaver, especially when I’m in a rush. But, it would be great if it showed recent transactions as well.",
    "The Quick Balance option is handy, but I've had security concerns. Maybe adding a simple passcode for this feature would be better.",
    "Really appreciate the Quick Balance feature for its convenience. However, the app sometimes crashes when I use this feature.",
    "Quick Balance is a great idea, but it’s not always accurate. It needs to sync better with recent transactions.",
    "I use Quick Balance all the time! It’s fast and easy. Just wish the app’s overall performance was as reliable.",
    "The Quick Balance feature is quite useful, but it doesn't always work when I have a poor internet connection, which is a bit annoying.",
    "Handy feature, but I accidentally enabled Quick Balance and now can't figure out how to turn it off. The app could use clearer settings.",
    "Love the convenience of Quick Balance! It's a great feature, but it doesn't always reflect pending transactions, which can be misleading.",
    "Quick Balance is a good feature, but the app logs me out too often, making it less convenient than it should be."
]


# Request New Card
sample_reviews_17 = [
    "Had to request a new card after misplacing mine. The process was pretty straightforward, but it took almost two weeks to get my card. Wish it was quicker.",
    "Requested a new card through the app. Easy to navigate, but it crashed a couple of times before completing the request. The card arrived in a week.",
    "The 'Request New Card' feature was a lifesaver after I lost my wallet. The new card arrived quickly, but I wish there were more design options.",
    "Tried requesting a new card, but the app kept giving an error. Had to call customer service, which took forever. The service could be better.",
    "The app made it really simple to request a new card, but I wasn't happy with the lack of updates on the delivery status. Got the card after 10 days.",
    "The new card request feature is convenient, but the app doesn't save your progress if you exit halfway. Had to start over, which was annoying.",
    "Used the app to request a new card after mine got stolen. The process was smooth, but I didn't like that I couldn't choose a different delivery address.",
    "Requesting a new card was easy, but the app didn't provide any information about how to activate it once it arrived. Ended up calling support.",
    "I appreciated how quickly I could request a new card through the app. However, the lack of a tracking feature for the card's delivery was disappointing.",
    "The app's request new card feature is intuitive, but the actual card took longer to arrive than promised. Good app, but delivery needs work."
]


# Rewards
sample_reviews_18 = [
    "I really appreciate the SCENE+ points feature in this app. It's great to see my rewards accumulate, but sometimes the app is slow to update my points.",
    "The rewards section is pretty straightforward. I like how it displays my points, but I wish it had more options to redeem them.",
    "Honestly, I'm a bit disappointed with the rewards feature. It often glitches and doesn't show my correct points balance.",
    "Love the rewards program! It's easy to track my SCENE+ points. However, the app crashes occasionally when I try to access the rewards section.",
    "The app does a decent job in showing my loyalty points, but it's not very user-friendly. I struggle to find how to redeem my points.",
    "Great concept with the SCENE+ points, but the execution could be better. The rewards section needs a more intuitive design.",
    "The rewards feature is a nice touch, but it often takes a long time to load my points, which is frustrating.",
    "I'm pleased with the rewards feature overall, but it would be helpful if it included more detailed information about how I can earn more points.",
    "The rewards feature is okay, but I've experienced some errors when trying to redeem my points. Needs improvement.",
    "I like the rewards feature, especially seeing my SCENE+ points grow. But, the app sometimes logs me out unexpectedly when I'm in the rewards section."
]


# Save and Share Statements
sample_reviews_19 = [
    "The 'Save and Share' feature is quite handy, but I noticed it sometimes crashes when I try to share my statement via email. Overall, a useful feature but needs stability improvements.",
    "I love how easily I can save my transaction statements, but I wish the app allowed me to customize the date range more flexibly. It's a bit limited in options right now.",
    "The sharing option is a lifesaver for quickly sending statements to my accountant. However, the file format is sometimes not compatible with other software which can be frustrating.",
    "Saving statements is straightforward, but the app lacks an option to save them in different formats like PDF or Excel. This would make it a lot more useful for my business needs.",
    "I appreciate the ability to save and share my bank statements directly from the app. However, the process is a bit slow and could be more user-friendly.",
    "The feature works well for the most part, but I've had issues with the app freezing when I try to share a statement with too many transactions. Might need some optimization.",
    "Very convenient for keeping track of my expenses. But, the lack of a search function within saved statements makes it hard to find specific transactions quickly.",
    "The 'Save and Share' feature is a great concept, but it needs an update. The resolution of shared statements is quite low, making them hard to read at times.",
    "I find saving statements easy and efficient. However, the sharing feature sometimes fails to attach the statement to emails, which can be quite annoying.",
    "Overall, a useful feature for managing finances, but the user interface for saving and sharing could be more intuitive. It takes a few tries to get used to it."
]

topics_importance_1 = [
    {"sequence": review, "label": "Application Performance"} for review in sample_reviews_01
]
topics_importance_2 = [
    {"sequence": review, "label": "Accessibility"} for review in sample_reviews_02
]
topics_importance_3 = [
    {"sequence": review, "label": "Appointment Booking"} for review in sample_reviews_03
]
topics_importance_4 = [
    {"sequence": review, "label": "Biometric Login"} for review in sample_reviews_04
]
topics_importance_5 = [
    {"sequence": review, "label": "Budgeting"} for review in sample_reviews_05
]
topics_importance_6 = [
    {"sequence": review, "label": "Chat"} for review in sample_reviews_06
]
topics_importance_7 = [
    {"sequence": review, "label": "Cheque Deposit"} for review in sample_reviews_07
]
topics_importance_8 = [
    {"sequence": review, "label": "Credit Score"} for review in sample_reviews_08
]
topics_importance_9 = [
    {"sequence": review, "label": "Email Money Transfer"} for review in sample_reviews_09
]
topics_importance_10 = [
    {"sequence": review, "label": "Errors"} for review in sample_reviews_10
]
topics_importance_11 = [
    {"sequence": review, "label": "Fee"} for review in sample_reviews_11
]
topics_importance_12 = [
    {"sequence": review, "label": "Info Alerts"} for review in sample_reviews_12
]
topics_importance_13 = [
    {"sequence": review, "label": "International Money Movement"} for review in sample_reviews_13
]
topics_importance_14 = [
    {"sequence": review, "label": "Investments"} for review in sample_reviews_14
]
topics_importance_15 = [
    {"sequence": review, "label": "Login and Logout Issues"} for review in sample_reviews_15
]
topics_importance_16 = [
    {"sequence": review, "label": "Quick Balance"} for review in sample_reviews_16
]
topics_importance_17 = [
    {"sequence": review, "label": "Request New Card"} for review in sample_reviews_17
]
topics_importance_18 = [
    {"sequence": review, "label": "Rewards"} for review in sample_reviews_18
]
topics_importance_19 = [
    {"sequence": review, "label": "Save and Share Statements"} for review in sample_reviews_19
]


In [58]:
combine_df = (
    topics_importance + topics_importance_1 + topics_importance_2 + topics_importance_3 +
    topics_importance_4 + topics_importance_5 + topics_importance_6 + topics_importance_7 +
    topics_importance_8 + topics_importance_9 + topics_importance_10 + topics_importance_11 +
    topics_importance_12 + topics_importance_13 + topics_importance_14 + topics_importance_15 +
    topics_importance_16 + topics_importance_17 + topics_importance_18 + topics_importance_19
)


In [59]:
df_few_shot_examples_2 = pd.DataFrame(combine_df)

In [60]:
review_2 = df_few_shot_examples_2['sequence'].to_list()
label_2 = df_few_shot_examples_2['label'].to_list()

In [61]:
clf.fit(review_2,label_2)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

In [62]:
filtered_reviews3['predicted_2'] = clf.predict(filtered_reviews3['Review'].to_list())

<ipython-input-62-3ef8c2b1523d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews3['predicted_2'] = clf.predict(filtered_reviews3['Review'].to_list())


In [69]:
filtered_reviews = filtered_reviews3[filtered_reviews3['predicted_2'] == 'Biometric Login']

In [70]:
filtered_reviews

,Review_ID,Date,Rating,Review_Language,Version,Review_Likes,Review,processed_review,word_count,predicted,GMM_Pred,predicted_2
5,5,2021-09-19 12:32:37,5,en,20.30.1,0,"Now I can use my fingerprints to log on, just ...",now i can use my fingerprints to log on just m...,15,relevant,18,Biometric Login
180,180,2021-06-13 09:40:05,1,en,20.27.0,0,Can't even get the app to open now let alone a...,cant even get the app to open now let alone ac...,15,relevant,11,Biometric Login
230,230,2021-04-24 07:05:24,1,en,20.25.1,0,What's going on with the app?? Fingerprint isn...,whats going on with the app fingerprint isnt w...,52,relevant,16,Biometric Login
428,428,2023-03-03 12:44:58,3,en,20.47.0,0,they took away the fingerprint scan access whi...,they took away the fingerprint scan access whi...,11,relevant,8,Biometric Login
644,644,2022-07-09 15:34:53,1,en,20.33.2,0,You can sign in with finger print,you can sign in with finger print,7,relevant,0,Biometric Login
...,...,...,...,...,...,...,...,...,...,...,...,...
8796,8796,2023-12-30 20:30:17,3,en,2311.0.1,17,My fingerprint doesn't work anymore suddenly. ...,my fingerprint doesnt work anymore suddenly ca...,60,relevant,16,Biometric Login
8870,8870,2022-04-28 11:52:49,1,en,20.37.2,8,Last two updates to this app have left it brok...,last two updates to this app have left it brok...,88,relevant,17,Biometric Login
8914,8914,2021-02-18 20:25:12,5,en,20.23.0,0,Work with finger printing,work with finger printing,4,relevant,18,Biometric Login
8989,8989,2021-10-26 07:18:13,1,en,20.31.1,9,New 2-step verification feature requires you t...,new 2step verification feature requires you to...,85,relevant,17,Biometric Login


In [71]:
filtered_reviews3.to_csv('/content/20_categories.csv', index=False)